In [8]:
import pandas as pd
import numpy as np
import pandas as pd
from datetime import datetime
import yfinance as yf

import warnings

try:

    df_list = []

    for i in range(1, 31):
        try:
            filename = f'resources/transactions_summary ({i}).csv'
            df = pd.read_csv(filename)
            df_list.append(df)
            print(f'Successfully loaded file {filename}')
            # Perform any data analysis or manipulation you need
        except FileNotFoundError:
            pass

    shake_data_df = pd.concat(df_list, ignore_index=True)
    shake_data_df['Date'] = pd.to_datetime(shake_data_df['Date']).dt.strftime('%Y-%m-%d')

except FileNotFoundError:
    print("File not found. Please check the filename and path and try again.")

Successfully loaded file resources/transactions_summary (1).csv
Successfully loaded file resources/transactions_summary (2).csv


In [10]:
# assuming you have a datetime column named "date" in the shake_data_df dataframe
start_date = '2021-10-01'
end_date = '2023-10-09'

date_mask = (shake_data_df['Date'] >= start_date) & (shake_data_df['Date'] <= end_date)
filtered_data = shake_data_df.loc[date_mask]

# Create a boolean mask for the desired values
mask2 = filtered_data['Transaction Type'].isin(['shakingstats', 'card cashbacks', 'shakesquads'])

# Apply the mask to the DataFrame to get the filtered rows
filtered_df2 = filtered_data[mask2]
# Group the filtered dataframe by 'Amount credited' and find the sum, and by 'Spot rate' to find the mean
grouped_df = filtered_df2.groupby('Transaction Type').agg({'Amount Credited': 'sum', 'Spot Rate': 'mean'})

# Print the resulting grouped dataframe
# Create a new column 'Value' that multiplies 'Amount credited' and 'Spot rate'
grouped_df['Value'] = grouped_df['Amount Credited'] * grouped_df['Spot Rate']

# Group the dataframe by 'Transaction Type' where 'Transaction Type' is equal to 'shakingsats'
shakingstats = shake_data_df[shake_data_df['Transaction Type'] == 'shakingsats']


# Group the dataframe by 'Transaction Type' where 'Transaction Type' is equal to 'shakingsats'
shakingstats = shake_data_df[shake_data_df['Transaction Type'] == 'shakingsats'].groupby('Transaction Type').agg({'Amount Credited': 'sum', 'Spot Rate': 'mean'})

# Print the new DataFrame
shakingstats['Value'] = shakingstats['Amount Credited'] * shakingstats['Spot Rate']
merged = pd.concat([grouped_df,shakingstats])

shake_data_df1 = filtered_data.drop(['Debit Currency','Direction', 'Source / Destination', 'Blockchain Transaction ID'], axis=1)
new_shake_data_df = shake_data_df1[(shake_data_df1['Transaction Type'] == 'purchase/sale') & (shake_data_df1['Credit Currency'] == 'BTC')]
with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    new_shake_data_df['weightedbuy'] = new_shake_data_df['Buy / Sell Rate'] * new_shake_data_df['Amount Debited']
weighted_average=new_shake_data_df['weightedbuy'].sum() / new_shake_data_df['Buy / Sell Rate'].mean()

buy_sell_rate = new_shake_data_df['Buy / Sell Rate'].mean()
stack = new_shake_data_df['Amount Credited'].sum()
cash = new_shake_data_df['Amount Debited'].sum()


## Group By BTC Total

# Filter the DataFrame based on Transaction Type and Credit Currency
BTC_Total = filtered_data[(filtered_data['Transaction Type'] == 'purchase/sale') & (filtered_data['Credit Currency'] == 'BTC')]
BTC_Total = BTC_Total[BTC_Total['Transaction Type'] == 'purchase/sale'].groupby('Transaction Type').agg({'Amount Debited': 'sum', 'Amount Credited': 'sum', 'Buy / Sell Rate': 'mean'})


## Current Price
import yfinance as yf
btc_df = yf.Ticker('BTC-CAD').history(period='7y',interval='1d',actions=False).reset_index()
btc_df = btc_df.iloc[-1].Close

## Stack

# get input from user and convert to float
# my_float_value = float(input("Please enter your float value: "))
my_float_value = BTC_Total.iloc[0,1]
# # multiply DataFrame by user input and store in Stack variable
Stack = my_float_value * btc_df

## Current Price

btc_df = yf.Ticker('BTC-CAD').history(period='7y',interval='1d',actions=False).reset_index()
Current_Price = btc_df.iloc[-1].Close

## Return
returndf = (Stack/cash)-1


print(f"From {start_date} to {end_date}.")
print(f"Current Price: {(Current_Price):.2f}")
print(f"Total Invested: {cash:.2f}")
print(f"Current Value: {(Stack):.4f}")
print(f"Current Return: {(returndf * 100):.2f}%")
print(f"Weighted Average: {weighted_average:.2f}")
print(f"Average DCA Price: {buy_sell_rate:.2f}")
print(f"The total value from card cashbacks is {merged.iloc[0,2]:.2f}")
print(f"The total value from card shakesquads is {merged.iloc[1,2]:.2f}")
print(f"The total value from card shakingsats is {merged.iloc[2,2]:.2f}")

From 2021-10-01 to 2023-10-09.
Current Price: 37280.23
Total Invested: 9067.87
Current Value: 7909.3260
Current Return: -12.78%
Weighted Average: 12896.15
Average DCA Price: 34528.45
The total value from card cashbacks is 733.64
The total value from card shakesquads is 22.41
The total value from card shakingsats is 134.72
